In [ ]:
from langchain.chains import ConversationalRetrievalChain
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.callbacks.streaming_stdout_final_only import (
    FinalStreamingStdOutCallbackHandler,
)
from langchain.agents import AgentType
from langchain.vectorstores import Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
import os
from dotenv import load_dotenv
import pinecone
load_dotenv()

OPENAI_API_KEY = ""
pinecone_key = os.getenv("PINECONE_KEY")
pinecone_env = os.getenv("PINECONE_ENV")


In [ ]:
pinecone.init(api_key=pinecone_key, environment=pinecone_env)
index = pinecone.Index('chagpt_text_model')

In [ ]:
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
vectordb = Pinecone(index, embedding_function=embeddings.embed_query, text_key="text")

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks.streaming_stdout_final_only import (
    FinalStreamingStdOutCallbackHandler,
)
llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    temperature=0,
    model_name = "gpt-3.5-turbo-16k",
    streaming=True,
    callbacks=[FinalStreamingStdOutCallbackHandler()],
)

In [ ]:
from langchain.chains import RetrievalQA

retriever = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectordb.as_retriever(),

)

In [ ]:
from langchain.agents import Tool
tool_desc = """Tool Description"""
tools = [Tool(
    func=retriever.run,
    description=tool_desc,
    name='tool name'
)]

In [ ]:
from langchain.memory import DynamoDBChatMessageHistory

chat_history = DynamoDBChatMessageHistory(table_name='conversation-store', session_id='qasc')

In [ ]:
from langchain.chains.conversation.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)
memory.chat_memory = chat_history

In [ ]:
from langchain.agents import initialize_agent
from langchain.agents import AgentType

conversational_agent = initialize_agent(
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    tools=tools,
    llm=llm,
    verbose=True,
    memory=memory,
)

In [ ]:
sys_pro = """system_message Description
"""
prompt = conversational_agent.agent.create_prompt(
    system_message=sys_pro,
    tools=tools
)
conversational_agent.agent.llm_chain.prompt = prompt

In [ ]:
memory.chat_memory.messages

In [ ]:
hello = conversational_agent.run("Hello")
print(hello)